# Audio Filter Design

More advanced filters needed in many real-life applications
- Equalization of loudspeakers/headphones
- 3D sound using HRTFs (head-related transfer function)
- Room correction (loudspeaker-room EQ)
- Digital musical instruments (e.g., piano soundboard, or violin body)
- Acoustic system modeling in general

## Intro & motivation
- Which application? What purpose?
    - Requirements help to choose the design method
- General principle
    - Quick methods for real-time applications
    - Accurate methods for off-line processing
- Things to remember in audio filter design
    - The frequency resolution of the human hearing system in nonlinear: good at low, poor at high frequencies
    - Conventional ”textbook” filter design techniques (lowpass, highpass, bandpass...) are usually unsuitable

## Properties of filters
### Convolution for linear and time-invariant (LTI) systems:
$$y [ n ] = h [ n ] * x [ n ] = \sum _ { k = - \infty } ^ { \infty } x [ k ] h [ n - k ]$$
where $h[n]$ is the filter’s impulse response and $x[n]$ is the input signal

### In the z-domain
$$Y ( z ) = H ( z ) X ( z ) \Leftrightarrow H ( z ) = \frac { Y ( z ) } { X ( z ) } = \frac { \sum _ { k = 0 } ^ { M } b _ { k } z ^ { - k } } { \sum _ { k = 0 } ^ { N } a _ { k } z ^ { - k } }$$

### Frequency response
$$H \left( e ^ { j \omega } \right) = \frac { Y \left( e ^ { j \omega } \right) } { X \left( e ^ { j \omega } \right) }$$
### Magnitude response
$$\left| H \left( e ^ { j \omega } \right) \right|$$
### Phase response: Radian phase shift
$$\theta ( \omega ) = \arg \left\{ H \left( e ^ { j \omega } \right) \right\}$$
### Phase delay: Time delay of each sinusoidal component (in samples)
$$P ( \omega ) = - \frac { \theta ( \omega ) } { \omega }$$
### Group delay: 
Time delay (in samples) of the amplitude envelope of a sinusoid at frequency $\omega$ 
$$D ( \omega ) = - \frac { d } { d \omega } \theta ( \omega )$$

-------------
## Phase Characteristics

- Linear phase
    - Easy to obtain using FIR filters 
    - Symmetric impulse response
    - Phase delay = group delay
    - Sometimes desirable, but why?
        - Constant phase delay(latency) at all frequencies
- Zero phase
    - Special case of a linear-phase filter
    - Impulse response symmetric w.r.t. 0 → noncausal!
- Minimum phase
    - No zeros outside the *unit circle*
    - Often desirable, why?
        - no pre echo
    - Releases energy as fast as possible for the given magnitude response
    - Always stable and causal
        - Also the inverse filter is stable!
- Maximum phase
    - All zeros outside the unit circle (or on the unit circle)     
    - Time-reversed version of the minimum-phase system

-----
## Filter design problem

- Approximate the target response with a digital filter
- Minimize the error function
    - $E \left( e ^ { j \omega } \right) = H \left( e ^ { j \omega } \right) - \hat { H } \left( e ^ { j \omega } \right)$ (Error = target - approximation)
- In digital filter design, $L_p$ norms are often used
    - $L_2$ leads to the least-squares design
    - $L _ { \infty }$ leads to the Chebyshev (minimax) design
    
Different norms used for different applications
### Least-Squares ($L_2$) Norm

### Chebyshev ($L _ { \infty }$ or minimax) Norm
- Minimize the maximum component of the error norm
- Need an iteration loop and an end rule (for selected accuracy)
- Finally the error function will be “equiripple”

$$E _ { \infty } \left( e ^ { j \omega } \right) = \left\| H \left( e ^ { j \omega } \right) - \hat { H } \left( e ^ { j \omega } \right) \right\| _ { \infty } \hat { = } \min _ { - \pi < \omega < \pi } \left| H \left( e ^ { j \omega } \right) - \hat { H } \left( e ^ { j \omega } \right) \right|$$

----------
## Auditory Resolution
- Take properties of human hearing into account
    - Linear or non-uniform (e.g. critical bands) frequency scale?
    - The domain where the magnitude is matched (e.g. T60-domain)?
- Tools
    - Auditory smoothing: averaging of the spectrum with a sliding window
    - Weighting functions: suppress less important frequency areas
    - Frequency warping: expand important frequency range; shrink the less important range

## Frequency Warping
Frequency scale warping before designing the filter
- Replace the unit delay with a first order allpass filter
- $\lambda$ is the warping parameter

$$z ^ { - 1 } \rightarrow \frac { z ^ { - 1 } - \lambda } { 1 - \lambda z ^ { - 1 } }$$

![freq_scale_warping](images/freq_scale_warping.png)

### Properties of Warped Filters
- Allpass filter → flat magnitude response
- Frequency-dependent group delay
- If $\lambda$ = 0.7661 ($f_s$ = 48 kHz), warping is set to match the Bark scale (Smith and Abel 1999)
- Design the filter in warped frequency domain → concentrate on the important frequency range

### How to Warp?

- Implement the filters in the warped frequency domain
     - WFIR or WIIR structures
     - More accuracy, but more computation as well
     - In WIIR structures the delay-free loops must be avoided → remap coefficients
- Unwarp the warped transfer function to a standard IIR filter

### Warped FIR Filter
![Warped_FIR_Filter](images/Warped_FIR_Filter.png)

- $s(n)$ is the signal to be warped
- $sw(n)$ is the warped signal
- The extent of warping is determined by $\lambda$
     - With $\lambda = 0$, the warped FIR filter becomes a standard FIR filter

-----

## FIR Filter Design
- Frequency sampling
    - Construct the target frequency response (magnitude & phase)
    - Also at negative frequencies: the same magnitude but opposite phase
    - Use IFFT to obtain the filter coefficients
    - The length of IFFT specifies the filter size
    - There will be ”don’t care” regions between the selected points
- Windowing method
    - Truncate the impulse response using a window function
    - Many choices for window functions!
    - Suitable for modeling measured/estimated impulse responses in audio

-----------

## Window Functions
### Rectangular window
- Minimizes the truncated time-domain L2 norm
- Gibb’s phenomenon (passband ripple)
- Low sidelobe rejection
- Relatively steep transition band
~ ![rectangular_window](images/rectangular_window.png)
    
### Other window functions 
   
~ ![hann](images/hann.png)
~ ![hamming](images/hamming.png)
~ ![blackman](images/blackman.png)
~ ![window_comparison](images/window_comparison.png)

-------------

## IIR Filter Design
1. Linear prediction 
2. Prony’s Method
3. Fixed-pole IIR filters
    - First select the poles, then optimize the feedforward coefficients

### Linear prediction 
- a.k.a. Linear Prediction Coding (LPC)
- Powerful analysis technique, especially in speech processing
- Other applications
    - Pitch prediction
    - Signal modeling for sound synthesis 
    - Detection of clicks in audio restoration 
    - Spectral envelope estimation
- Predict the next sample as a linear combination of previous samples

#### LPC coefficients
Compute the next sample as a linear combination of the previous samples
$$\hat { x } ( n ) = \sum _ { k = 1 } ^ { P } a _ { k } x ( n - k )$$
Minimize the expected value of the mean-squared prediction error
$$E \left\{ e _ { n } ^ { 2 } \right\} = E \left\{ ( x ( n ) - \hat { x } ( n ) ) ^ { 2 } \right\}$$

#### LPC model
- Analysis
     - Use the autocorrelation or covariance method to obtain prediction coefficients
     - FIR filter with prediction coefficients is an inverse filter, which whitens the input signal (decorrelation)
     - Autocorrelation method → minimum phase
- Synthesis
     - Prediction coefficients used in an all-pole IIR filter yield a filter with the spectral characteristics of the input signal
     
### Prony’s Method
- Design IIR filter from a given impulse response
- Solve denominator coefficients **using linear prediction**
- Solve numerator coefficients by matching the $N +1$ first samples of the impulse response

------
## Conclusions
- Properties of filters: magnitude and phase
     - The phase delay and group delay are identical for linear-phase systems but otherwise different
- Various audio filter design methods
     - For FIR filters: Least squares, weighted least squares, frequency sampling, impulse response truncation/windowing
     - For IIR filters: linear prediction, Prony’s method
- Auditory resolution can be taken into account
     - Weighting or warping
- The suitable method depends on the application
- Typical design problems are HRTF filters, musical instruments, and equalizers